The equations that we are trying solve are those of the paper titled "Diffuse-charge dynamics in electrochemical systems" by Bazant, Thornton and Ajdari published in https://journals.aps.org/pre/abstract/10.1103/PhysRevE.70.021506

In [1]:
from fipy import *
m = Grid1D(nx=100,Lx=2.0)
v = 1
epsilon= 0.05
delta = 0.1
x = m.cellCenters[0]

The set of equations is 

\begin{equation}
\begin{split}
\frac{\partial c}{\partial t}
&= \varepsilon\frac{\partial}{\partial x} \left(\frac{\partial c}{\partial x} + \rho\frac{\partial \phi}{\partial x}\right)\\
\frac{\partial \rho}{\partial t} 
&=  \varepsilon\frac{\partial}{\partial x} \left(\frac{\partial\rho}{\partial x}  + c \frac{\partial \phi}{\partial x}\right)\\
-\varepsilon^2 \frac{\partial^2\phi}{\partial x^2} &= \rho
\end{split}
\end{equation}

With the initial conditions

\begin{equation}
\begin{split}
c(x,0) &= 1\\
\rho(x,0) &= 0\\
\phi(x,0) &= v(x-1)\\
\end{split}
\end{equation}


and the boundary conditions

\begin{equation}
\begin{split}
\frac{\partial c}{\partial x} + \rho\frac{\partial \phi}{\partial x} &= 0\\
\frac{\partial \rho}{\partial x} + c\frac{\partial \phi}{\partial x} &= 0\\
\text{where both fluxes are zero at} &\quad\text{$x = 0$ and $x=2$}\\
\end{split}
\end{equation}

\begin{equation}
\phi = +v - \varepsilon\delta \frac{\partial \phi}{\partial x} \quad\text{at $x = 2$}
\end{equation}

\begin{equation}
\phi = -v + \varepsilon\delta \frac{\partial \phi}{\partial x} \quad\text{at $x = 0$}
\end{equation}

In [5]:
##Initial conditions
c = CellVariable(name="c", mesh=m , value=1.0)

rho = CellVariable(name="rho", mesh=m , value=0.0)

phi = CellVariable(name="phi",mesh=m)
phi.setValue(v*(x-1), where=m.cellCenters)

#vi =Viewer((phi, c, rho),datamin=-1.1, datamax=1.1)

In [6]:
##Boundary conditions
c.faceGrad.constrain(-rho.faceValue*phi.faceGrad, where=m.facesLeft)
c.faceGrad.constrain(-rho.faceValue*phi.faceGrad, where=m.facesRight)

rho.faceGrad.constrain(-c.faceValue*phi.faceGrad, where=m.facesLeft)
rho.faceGrad.constrain(-c.faceValue*phi.faceGrad, where=m.facesRight)

phi.faceGrad.constrain((phi.faceValue+v)/(epsilon*delta), where=m.facesLeft)
phi.faceGrad.constrain((phi.faceValue-v)/(-epsilon*delta), where=m.facesRight)

In [ ]:
##Equation

Drho = epsilon*rho
Dc   = epsilon*c

eq1 = TransientTerm(var=c) == DiffusionTerm(coeff=epsilon, var=c) +DiffusionTerm(coeff=Drho, var=phi)
eq2 = TransientTerm(var=rho) == DiffusionTerm(coeff=epsilon, var=rho) +DiffusionTerm(coeff=Dc, var=phi)
eq3 = DiffusionTerm(coeff=epsilon*epsilon, var=phi)+ImplicitSourceTerm(coeff=1.0, var=rho) == 0
eqns = eq1 & eq2 & eq3 

In [ ]:
##Solve
#vi =Viewer((phi, c, rho),datamin=-1.1, datamax=1.1)
#from builtins import range

res = 1e+10
restol= 1e-13

for t in range(100):
    c.updateOld()
    rho.updateOld()
    phi.updateOld()
    while res > restol:
        res = eqns.sweep(dt=1e-15)
        print(res)
    
#if __name__=='__main__':
#        vi.plot()
#        raw_input("No-flux -- stationary. Press <return> to proceed..")


However the solution method above gave some errors, originated with the warning "self.factorizeFnc(*args, **kwargs)" 